In [3]:
"""References
Ghiaus, C.
Causality issue in the heat balance method for calculating the design heating and cooling load
Energy, 2013, 50, 292 - 301

Raillon, L.
Experimental Identification of Physical Thermal Models for Demand Response and Performance
Evaluation, https://www.researchgate.net/publication/326058713_
"""
from sympy import *

init_printing(use_unicode=True)

# Nodes without thermal capacity must be placed on the left
Ro, Ri, Cw, Ci, Aw, Ai = symbols('Ro, Ri, Cw, Ci, Aw, Ai')
s = symbols('s')  # Laplace operator

In [4]:
R = diag(Ro, Ri)
C = diag(Cw, Ci)

A = Matrix([[1, 0], [-1, 1]])  # Incidence matrix
nu = 3  # Number of inputs, e.g. Outdoor temperature, Solar irradiance, Heating power
u = [True, False, True, True]  # Mask for removing temperature or heat flux sources set to zero

nb = R.shape[0]  # Number of branches
nn = C.shape[0]  # Number of nodes
nz = 0  # Number of nodes without thermal capacity
for i in range(nn):
    if C[i, i] == 0:
        nz += 1

G = R.inv()
Cc = C[nz:, nz:]
nx = Cc.shape[0]  # Number of states

In [5]:
# Build state-space model
K = -A.T @ G @ A
K11 = K[:nz, :nz]
K12 = K[:nz, nz:]
K21 = K[nz:, :nz]
K22 = K[nz:, nz:]

Kb = A.T @ G
Kb1 = Kb[:nz, :]
Kb2 = Kb[nz:, :]

As = simplify(Cc.inv() @ (-K21 @ K11.inv() @ K12 + K22))
Bs0 = Cc.inv() @ (-K21 @ K11.inv() @ Kb1 + Kb2)
Bs1 = Cc.inv() @ -K21 @ K11.inv()
Bs2 = Cc.inv()
if Bs1.is_zero:
    Bs = Matrix(BlockMatrix([[Bs0, Bs2]]).as_explicit())
else:
    Bs = Matrix(BlockMatrix([[Bs0, Bs1, Bs2]]).as_explicit())

Cs = simplify(-K11.inv() @ K12)
ny = Cs.shape[0]
Ds0 = simplify(-K11.inv() @ Kb1)
Ds1 = simplify(-K11.inv())
Ds2 = zeros(ny, nu)
Ds = Matrix(BlockMatrix([[Ds0, Ds1, Ds2]]).as_explicit())

Bs = simplify(Bs[:, u])
if not Ds.is_zero:
    Ds = simplify(Ds[:, u])

# Additional parameters of the input matrix
Bs2 = zeros(nx, nu)
Bs2[0, 0] = Bs[0, 0]
Bs2[0, 1] = Bs[0, 1] * Aw
Bs2[1, 1] = Bs[1, 2] * Ai
Bs2[1, 2] = Bs[1, 2]
Bs = simplify(Bs2)


In [6]:
# Check observability and structural identifiability for MISO LTI system
Cs = Matrix([0, 1]).T
sI = s * eye(nx)

# Check the observability of the system
Obs = [Cs]
for i in range(1, nx):
    Obs.append(Cs @ As ** i)

if not Matrix(Obs).rank() == nx:
    raise ValueError('The model is not observable')

eqs = []
for i in range(nu):
    H = collect(cancel(simplify(Cs @ (sI - As).inv() @ Bs[:, i]))[0], s)
    n, d = fraction(H)
    eqs += [c for c in Poly(n, s).coeffs() + Poly(d, s).coeffs() if c != 1]

eqs = list(set(eqs))
rc_par = list(set(list(As.free_symbols) + list(Bs.free_symbols)))
if not len(rc_par) == len(eqs):
    raise ValueError('The model is not structurally identifiable')

sys_eq = []
for i in range(len(eqs)):
    sys_eq += [Eq(eqs[i], symbols('x%d' % i))]

print('Solution')
print('-' * 8)
results = solve(sys_eq, rc_par, dict=True)[0]
for k, v in results.items():
    print(f'{k}: {v}')


Solution
--------
Ci: x5/x3
Ro: (x0*x1*x3 - x1**2*x5 - x3**2)/(x0*x3 - x1*x5)
Cw: (x0*x3 - x1*x5)**2/(x3*(x0*x1*x3 - x1**2*x5 - x3**2))
Ri: x3**2/(x0*x3 - x1*x5)
Ai: x4/x3
Aw: -(x0*x3 - x1*x5)*(x1*x4 - x2*x3)/(x3*(x0*x1*x3 - x1**2*x5 - x3**2))
